In [20]:
import os

In [21]:
%pwd

'e:\\Diabetes_classifier'

In [5]:
os.chdir('../')

In [22]:
%pwd

'e:\\Diabetes_classifier'

In [24]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class Data_TransformationConfig:
    root_dir:Path
    preprocessor_dir:Path

In [10]:
from src.DIABETES_CLASSIFIER.utils.common import read_yaml,create_directories
from src.DIABETES_CLASSIFIER.constants import config_file_path,param_file_path
from src.DIABETES_CLASSIFIER import logger

In [25]:

class Configuratoration:
    def __init__(self,config=config_file_path,params=param_file_path):
        self.config=read_yaml(config)
        self.params=read_yaml(params)

        create_directories([self.config.artifacts_root])

    def get_data_transformation(self) ->Data_TransformationConfig:
        config=self.config.data_transformation
        ingestion=self.config.data_ingestion

        create_directories([config.root_dir])

        transformation=Data_TransformationConfig(
            root_dir=config.root_dir,
            preprocessor_dir=config.preprocessor_dir)

        return (transformation, ingestion)
    



In [26]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,StandardScaler
import pandas as pd


In [27]:
from src.DIABETES_CLASSIFIER.utils.common import save_model

In [29]:
class Data_Transformation:
    def __init__(self,config,ingestion):
        self.config=config
        self.ingestion=ingestion
       

    def get_preprocessor_object(self):

        numerical_features=['age','hypertension','heart_disease','bmi','HbA1c_level','blood_glucose_level',]

        cat_features=['gender','smoking_history']

        logger.info("creation of preprocessor_obj")


        #building pipline
        num_pip=Pipeline(steps=[
            ('scaler',StandardScaler()),
            ('impute',SimpleImputer(strategy='mean'))
                ])

        cat_pip=Pipeline(steps=[
                ('coder',OneHotEncoder()),
                ('impuet',SimpleImputer(strategy='most_frequent'))
                ])
        
        preprocessor=ColumnTransformer([
            ('num_pip',num_pip,numerical_features),
            ('cat_pip',cat_pip,cat_features)
           ])


        return preprocessor
    
    def data_tranformation_initiated(self):

        train_df=pd.read_csv(self.ingestion.train_dir)

        test_df=pd.read_csv(self.ingestion.test_dir)

 
        x_train,x_test=(train_df.iloc[:,:-1],test_df.iloc[:,:-1])
        y_train,y_test=(train_df.iloc[:,-1],test_df.iloc[:,-1])


        preprocessor_obj=self.get_preprocessor_object()

        x_train_array=preprocessor_obj.fit_transform(x_train)

        x_test_array=preprocessor_obj.transform(x_test)

        save_model(obj=preprocessor_obj,file_path=self.config.preprocessor_dir)


        return (x_train_array,x_test_array,y_train,y_test)


In [31]:
try:
    data=Configuratoration()

    config,ingestion=data.get_data_transformation()

    transformation=Data_Transformation(config,ingestion)

    x_train,x_test,y_train,y_test=transformation.data_tranformation_initiated()

    print(x_train.shape)
    print(x_test.shape)
    print(y_train.shape)
    print(y_test.shape)

except Exception as e:
    raise e

[2023-08-21 20:24:02,736: INFO: common: safe loaded config\config.yaml]
[2023-08-21 20:24:02,744: INFO: common: safe loaded params.yaml]
[2023-08-21 20:24:02,748: INFO: common: The directory created]
[2023-08-21 20:24:02,752: INFO: common: The directory created]


[2023-08-21 20:24:03,072: INFO: 735389131: creation of preprocessor_obj]
(72109, 11)
(24037, 11)
(72109,)
(24037,)
